# 读取图

In [5]:
import py2neo
py2neo.__version__

'2021.2.3'

In [57]:
from py2neo import Node, Relationship, Graph, NodeMatcher, RelationshipMatcher

# 打开数据库
graph = Graph("bolt://localhost:7687", auth=("neo4j", "kzldeneo4j@"))
graph

Graph('bolt://localhost:7687')

In [58]:
cypher_="MATCH (n) RETURN count(n)"
graph.run(cypher_)

count(n)
2427


In [30]:
cypher_="MATCH (p) RETURN count(p)"
graph.run(cypher_)

count(p)
2427


In [4]:
cypher_="MATCH p=()-->() RETURN count(p)"
graph.run(cypher_)


count(p)
26507


### 查询节点类型

In [9]:
cypher_="call db.labels"
node_types = graph.run(cypher_)
node_types=node_types.to_data_frame()["label"]
node_types=node_types.values
node_types

array(['Patient', 'Syndrome', 'Sex', 'PastHistory', 'SoundofTinnitus',
       'SleepStatus', 'CoatedTongue', 'PulseManifestation',
       'DiagnosisofWesternMedicine', 'IncitingFactors', 'Tone25',
       'Symptom', 'TongueQuality', 'ConditionOfPersistence',
       'SolarTermOfOnset', 'TinnitusFrequency'], dtype=object)

### 查询关系类型

In [10]:
cypher_="call db.relationshipTypes"
relationship_types = graph.run(cypher_)
relationship_types=relationship_types.to_data_frame()["relationshipType"]
relationship_types=relationship_types.values
relationship_types

array(['sex_is', 'has_past_history', 'manifestation_of', 'has_symptom',
       'recommend_music', 'has_syndrome', 'diagnose_with',
       'has_inducement', 'part_of', 'onset', 'has_tinnitus_frequency'],
      dtype=object)

In [10]:
#训练集（有标签：中医辨证）
cypher_="MATCH (n) RETURN (n) "
graph.run(cypher_)

n
(_285:Sex {name: 'Sex'})
(_286:Sex {name: '\u7537'})
(_287:Sex {name: '\u5973'})


## 创建图投影

### 节点toString

In [11]:
import re
node_typesStr=str(node_types)
node_typesStr=re.sub(r' ',',',node_typesStr)
node_typesStr=re.sub(r'\n','',node_typesStr)
node_typesStr

"['Patient','Syndrome','Sex','PastHistory','SoundofTinnitus','SleepStatus','CoatedTongue','PulseManifestation','DiagnosisofWesternMedicine','IncitingFactors','Tone25','Symptom','TongueQuality','ConditionOfPersistence','SolarTermOfOnset','TinnitusFrequency']"

### 关系toString

In [12]:
import re
rel_typeStr=str(relationship_types)
rel_typeStr=re.sub(r' ',',',rel_typeStr)
rel_typeStr=re.sub(r'\n','',rel_typeStr)
rel_typeStr

"['sex_is','has_past_history','manifestation_of','has_symptom','recommend_music','has_syndrome','diagnose_with','has_inducement','part_of','onset','has_tinnitus_frequency']"

### 无向关系toString

In [13]:
rel_typeStr1=''
for item in relationship_types:
    relStr=f'{item}'+": {orientation: 'UNDIRECTED'}"
    # print(relStr)
    rel_typeStr1=rel_typeStr1+relStr+","
rel_typeStr1="{"+rel_typeStr1+"}"
rel_typeStr1=re.sub(r',}','}',rel_typeStr1)
rel_typeStr1

"{sex_is: {orientation: 'UNDIRECTED'},has_past_history: {orientation: 'UNDIRECTED'},manifestation_of: {orientation: 'UNDIRECTED'},has_symptom: {orientation: 'UNDIRECTED'},recommend_music: {orientation: 'UNDIRECTED'},has_syndrome: {orientation: 'UNDIRECTED'},diagnose_with: {orientation: 'UNDIRECTED'},has_inducement: {orientation: 'UNDIRECTED'},part_of: {orientation: 'UNDIRECTED'},onset: {orientation: 'UNDIRECTED'},has_tinnitus_frequency: {orientation: 'UNDIRECTED'}}"

### 训练集投影

In [6]:
cypher_="CALL gds.graph.project('tinnitusTraingraph01',"+node_typesStr+","+rel_typeStr+")"
cypher_

"CALL gds.graph.project('tinnitusAllgraph01',['Patient','Syndrome','Sex','PastHistory','ConditionofPersistence','SoundofTinnitus','SleepStatus','CoatedTongue','PulseManifestation','DiagnosisofWesternMedicine','IncitingFactors','Tone25','Symptom','TongueQuality'],['sex_is','has_past_history','manifestation_of','has_symptom','recommend_music','has_syndrome','diagnose_with','has_inducement','part_of','同证型患者'])"

### 全节点全关系有权重有向投影图

In [8]:
rel_typeStr

"['sex_is','has_past_history','manifestation_of','has_symptom','recommend_music','has_syndrome','diagnose_with','has_inducement','part_of']"

In [11]:
cypher_="CALL gds.graph.project('5foldPatientDirectedGraph02','Patient',"+rel_typeStr+",{relationshipProperties: 'weight'})"
graph.run(cypher_)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Patient: {label: 'Patient', properties: {}}}","{has_symptom: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_symptom', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, recommend_music: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'recommend_music', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, diagnose_with: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'diagnose_with', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, sex_is: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'sex_is', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, has_inducement: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_inducement', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, has_past_history: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_past_history', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, manifestation_of: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'manifestation_of', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, has_syndrome: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_syndrome', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, part_of: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'part_of', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",5foldPatientDirectedGraph02,595,0,18


In [14]:
#五折5foldAllDirectedGraph01、5foldAllDirectedGraph02、5foldAllDirectedGraph03
cypher_="CALL gds.graph.project('5foldNoSameWeightDirectedGraph01',"+node_typesStr+","+rel_typeStr+",{relationshipProperties: 'weight'})"
graph.run(cypher_)
cypher_

"CALL gds.graph.project('5foldNoSameWeightDirectedGraph01',['Patient','Syndrome','Sex','PastHistory','SoundofTinnitus','SleepStatus','CoatedTongue','PulseManifestation','DiagnosisofWesternMedicine','IncitingFactors','Tone25','Symptom','TongueQuality','ConditionOfPersistence','SolarTermOfOnset','TinnitusFrequency'],['sex_is','has_past_history','manifestation_of','has_symptom','recommend_music','has_syndrome','diagnose_with','has_inducement','part_of','onset','has_tinnitus_frequency'],{relationshipProperties: 'weight'})"

In [17]:
cypher_="CALL gds.graph.project('dataAllWeightDirectedGraph',"+node_typesStr+","+rel_typeStr+",{relationshipProperties: 'weight'})"
cypher_

"CALL gds.graph.project('dataAllWeightDirectedGraph',['Patient','Syndrome','Sex','PastHistory','SoundofTinnitus','SleepStatus','CoatedTongue','PulseManifestation','DiagnosisofWesternMedicine','IncitingFactors','Tone25','Symptom','TongueQuality','ConditionOfPersistence','SolarTermOfOnset'],['sex_is','has_past_history','manifestation_of','has_symptom','recommend_music','has_syndrome','diagnose_with','has_inducement','part_of'],{relationshipProperties: 'weight'})"

In [105]:
graph.run(cypher_)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{IncitingFactors: {label: 'IncitingFactors', properties: {}}, Sex: {label: 'Sex', properties: {}}, TongueQuality: {label: 'TongueQuality', properties: {}}, PastHistory: {label: 'PastHistory', properties: {}}, CoatedTongue: {label: 'CoatedTongue', properties: {}}, Patient: {label: 'Patient', properties: {}}, DiagnosisofWesternMedicine: {label: 'DiagnosisofWesternMedicine', properties: {}}, SleepStatus: {label: 'SleepStatus', properties: {}}, Syndrome: {label: 'Syndrome', properties: {}}, ConditionOfPersistence: {label: 'ConditionOfPersistence', properties: {}}, Tone25: {label: 'Tone25', properties: {}}, SoundofTinnitus: {label: 'SoundofTinnitus', properties: {}}, PulseManifestation: {label: 'PulseManifestation', properties: {}}, SolarTermOfOnset: {label: 'SolarTermOfOnset', properties: {}}, Symptom: {label: 'Symptom', properties: {}}}","{has_symptom: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_symptom', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, recommend_music: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'recommend_music', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, diagnose_with: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'diagnose_with', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, sex_is: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'sex_is', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, has_inducement: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_inducement', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, has_past_history: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_past_history', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, manifestation_of: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'manifestation_of', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, has_syndrome: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_syndrome', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, part_of: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'part_of', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",5foldAllDirectedGraph01,727,7893,27


### 全节点全关系无权重有向投影图

In [ ]:
#同证型患者、中医辨证属性数字
cypher_="CALL gds.graph.project('5foldAllnoweightNumberSyndromeDirectedGraph04',"+node_typesStr+","+rel_typeStr+")"
graph.run(cypher_)
cypher_

In [345]:
#五折5foldAllDirectedGraph01、5foldAllDirectedGraph02、5foldAllDirectedGraph03、5foldAllnoweightDirectedGraph04、5foldAllnoweightDirectedGraph02
cypher_="CALL gds.graph.project('5foldAllnoweightDirectedGraph05',"+node_typesStr+","+rel_typeStr+")"
print(cypher_)
graph.run(cypher_)


CALL gds.graph.project('5foldAllnoweightDirectedGraph05',['Patient','Syndrome','Sex','PastHistory','SoundofTinnitus','SleepStatus','CoatedTongue','PulseManifestation','DiagnosisofWesternMedicine','IncitingFactors','Tone25','Symptom','TongueQuality','ConditionOfPersistence','SolarTermOfOnset'],['sex_is','has_past_history','manifestation_of','has_symptom','recommend_music','has_syndrome','diagnose_with','has_inducement','part_of','同证型患者','onset'])


nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{IncitingFactors: {label: 'IncitingFactors', properties: {}}, Sex: {label: 'Sex', properties: {}}, TongueQuality: {label: 'TongueQuality', properties: {}}, PastHistory: {label: 'PastHistory', properties: {}}, CoatedTongue: {label: 'CoatedTongue', properties: {}}, Patient: {label: 'Patient', properties: {}}, DiagnosisofWesternMedicine: {label: 'DiagnosisofWesternMedicine', properties: {}}, SleepStatus: {label: 'SleepStatus', properties: {}}, Syndrome: {label: 'Syndrome', properties: {}}, ConditionOfPersistence: {label: 'ConditionOfPersistence', properties: {}}, Tone25: {label: 'Tone25', properties: {}}, SoundofTinnitus: {label: 'SoundofTinnitus', properties: {}}, PulseManifestation: {label: 'PulseManifestation', properties: {}}, SolarTermOfOnset: {label: 'SolarTermOfOnset', properties: {}}, Symptom: {label: 'Symptom', properties: {}}}","{has_symptom: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_symptom', properties: {}}, recommend_music: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'recommend_music', properties: {}}, 同证型患者: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: '\u540c\u8bc1\u578b\u60a3\u8005', properties: {}}, diagnose_with: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'diagnose_with', properties: {}}, sex_is: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'sex_is', properties: {}}, has_inducement: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_inducement', properties: {}}, has_past_history: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_past_history', properties: {}}, manifestation_of: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'manifestation_of', properties: {}}, has_syndrome: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_syndrome', properties: {}}, onset: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'onset', properties: {}}, part_of: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'part_of', properties: {}}}",5foldAllnoweightDirectedGraph05,728,11895,18


In [7]:
#tinnitusAllgraph01
cypher_="CALL gds.graph.project('dataAllgraph01',"+node_typesStr+","+rel_typeStr+")"
graph.run(cypher_)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{IncitingFactors: {label: 'IncitingFactors', properties: {}}, Sex: {label: 'Sex', properties: {}}, TongueQuality: {label: 'TongueQuality', properties: {}}, PastHistory: {label: 'PastHistory', properties: {}}, CoatedTongue: {label: 'CoatedTongue', properties: {}}, Patient: {label: 'Patient', properties: {}}, ConditionofPersistence: {label: 'ConditionofPersistence', properties: {}}, DiagnosisofWesternMedicine: {label: 'DiagnosisofWesternMedicine', properties: {}}, SleepStatus: {label: 'SleepStatus', properties: {}}, Syndrome: {label: 'Syndrome', properties: {}}, Tone25: {label: 'Tone25', properties: {}}, SoundofTinnitus: {label: 'SoundofTinnitus', properties: {}}, PulseManifestation: {label: 'PulseManifestation', properties: {}}, Symptom: {label: 'Symptom', properties: {}}}","{has_symptom: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_symptom', properties: {}}, recommend_music: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'recommend_music', properties: {}}, 同证型患者: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: '\u540c\u8bc1\u578b\u60a3\u8005', properties: {}}, diagnose_with: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'diagnose_with', properties: {}}, sex_is: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'sex_is', properties: {}}, has_inducement: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_inducement', properties: {}}, has_past_history: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_past_history', properties: {}}, manifestation_of: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'manifestation_of', properties: {}}, has_syndrome: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_syndrome', properties: {}}, part_of: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'part_of', properties: {}}}",dataAllgraph01,703,82787,30


### 全节点全关系有权重无向投影图

In [40]:
rel_typeStr1

"{sex_is: {orientation: 'UNDIRECTED'},has_past_history: {orientation: 'UNDIRECTED'},manifestation_of: {orientation: 'UNDIRECTED'},has_symptom: {orientation: 'UNDIRECTED'},recommend_music: {orientation: 'UNDIRECTED'},has_syndrome: {orientation: 'UNDIRECTED'},diagnose_with: {orientation: 'UNDIRECTED'},has_inducement: {orientation: 'UNDIRECTED'},part_of: {orientation: 'UNDIRECTED'},}"

In [16]:
#五折5foldAllUndirectedGraph01、5foldAllUndirectedGraph02
cypher_="CALL gds.graph.project('5foldNoSameUndirectedGraph01',"+node_typesStr+","+rel_typeStr1+",{relationshipProperties: 'weight'})"
graph.run(cypher_)
cypher_

"CALL gds.graph.project('5foldNoSameUndirectedGraph01',['Patient','Syndrome','Sex','PastHistory','SoundofTinnitus','SleepStatus','CoatedTongue','PulseManifestation','DiagnosisofWesternMedicine','IncitingFactors','Tone25','Symptom','TongueQuality','ConditionOfPersistence','SolarTermOfOnset','TinnitusFrequency'],{sex_is: {orientation: 'UNDIRECTED'},has_past_history: {orientation: 'UNDIRECTED'},manifestation_of: {orientation: 'UNDIRECTED'},has_symptom: {orientation: 'UNDIRECTED'},recommend_music: {orientation: 'UNDIRECTED'},has_syndrome: {orientation: 'UNDIRECTED'},diagnose_with: {orientation: 'UNDIRECTED'},has_inducement: {orientation: 'UNDIRECTED'},part_of: {orientation: 'UNDIRECTED'},onset: {orientation: 'UNDIRECTED'},has_tinnitus_frequency: {orientation: 'UNDIRECTED'}},{relationshipProperties: 'weight'})"

In [ ]:
cypher_="CALL gds.graph.project('dataAllWeightgraph',"+node_typesStr+","+rel_typeStr1+")"
graph.run(cypher_)

### 全节点全关系无向投影图

In [22]:

cypher_="CALL gds.graph.project('dataAllgraph02',"+node_typesStr+","+rel_typeStr1+")"
graph.run(cypher_)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{IncitingFactors: {label: 'IncitingFactors', properties: {}}, Sex: {label: 'Sex', properties: {}}, TongueQuality: {label: 'TongueQuality', properties: {}}, PastHistory: {label: 'PastHistory', properties: {}}, CoatedTongue: {label: 'CoatedTongue', properties: {}}, Patient: {label: 'Patient', properties: {}}, ConditionofPersistence: {label: 'ConditionofPersistence', properties: {}}, DiagnosisofWesternMedicine: {label: 'DiagnosisofWesternMedicine', properties: {}}, SleepStatus: {label: 'SleepStatus', properties: {}}, Syndrome: {label: 'Syndrome', properties: {}}, Tone25: {label: 'Tone25', properties: {}}, SoundofTinnitus: {label: 'SoundofTinnitus', properties: {}}, PulseManifestation: {label: 'PulseManifestation', properties: {}}, Symptom: {label: 'Symptom', properties: {}}}","{has_symptom: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'has_symptom', properties: {}}, recommend_music: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'recommend_music', properties: {}}, 同证型患者: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: '\u540c\u8bc1\u578b\u60a3\u8005', properties: {}}, diagnose_with: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'diagnose_with', properties: {}}, sex_is: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'sex_is', properties: {}}, has_inducement: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'has_inducement', properties: {}}, has_past_history: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'has_past_history', properties: {}}, manifestation_of: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'manifestation_of', properties: {}}, has_syndrome: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'has_syndrome', properties: {}}, part_of: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'part_of', properties: {}}}",dataAllgraph02,703,165574,32


### 只有Patient节点、has_syndrome无向关系投影图


In [14]:
tinnitusAllgraph
cypher_="CALL gds.graph.project('tinnitusAllgraph','Patient',{has_syndrome: {orientation: 'UNDIRECTED'}})"
graph.run(cypher_)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Patient: {label: 'Patient', properties: {}}}","{has_syndrome: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'has_syndrome', properties: {}}}",tinnitusAllgraph,595,0,3774


# 社区检测

## Louvain算法

### 运行Louvain算法

In [ ]:
#参数
'''
includeIntermediateCommunities：中间曾分配到的社区
relationshipWeightProperty: 'weight'
'''

In [19]:
#有权重
cypher_="CALL gds.louvain.stream('dataAllWeightDirectedGraph',{includeIntermediateCommunities: true, relationshipWeightProperty: 'weight'}) YIELD nodeId, communityId, intermediateCommunityIds RETURN gds.util.asNode(nodeId).name AS name, communityId, intermediateCommunityIds ORDER BY name ASC"
data_Louvain=graph.run(cypher_).to_data_frame()
data_Louvain

,name,communityId,intermediateCommunityIds
0,CoatedTongue,117,[117]
1,ConditionOfPersistence,42,[42]
2,DiagnosisofWesternMedicine,104,[104]
3,IncitingFactors,28,[28]
4,PastHistory,34,[34]
...,...,...,...
722,风热侵袭,77,[77]
723,饮酒,41,[41]
724,高脂血症,37,[37]
725,高血压,35,[35]


In [3]:
#tinnitusAllgraph01
cypher_="CALL gds.louvain.stream('dataAllgraph01',{includeIntermediateCommunities: true }) YIELD nodeId, communityId, intermediateCommunityIds RETURN gds.util.asNode(nodeId).name AS name, communityId, intermediateCommunityIds ORDER BY name ASC"
data_Louvain=graph.run(cypher_).to_data_frame()
data_Louvain

,name,communityId,intermediateCommunityIds
0,CoatedTongue,52,"[52, 52]"
1,ConditionofPersistence,52,"[52, 52]"
2,DiagnosisofWesternMedicine,52,"[81, 52]"
3,IncitingFactors,52,"[51, 52]"
4,PastHistory,52,"[52, 52]"
...,...,...,...
698,风热侵袭,52,"[50, 52]"
699,饮酒,52,"[52, 52]"
700,高脂血症,52,"[50, 52]"
701,高血压,52,"[52, 52]"


### 保存Louvain算法运行结果

In [20]:
#dataAll_Louvain
data_Louvain.to_csv("./outData/dataAll_Louvain权重.csv")

## leiden算法

### 运行leiden算法

In [ ]:
#有权重


In [23]:
#无权重
#tinnitusAllgraph01
cypher_="CALL gds.beta.leiden.stream('dataAllgraph02',{includeIntermediateCommunities: true }) YIELD nodeId, communityId, intermediateCommunityIds RETURN gds.util.asNode(nodeId).name AS name, communityId, intermediateCommunityIds ORDER BY name ASC"
data_leiden=graph.run(cypher_).to_data_frame()
data_leiden

,name,communityId,intermediateCommunityIds
0,CoatedTongue,373,[373]
1,ConditionofPersistence,373,[373]
2,DiagnosisofWesternMedicine,373,[373]
3,IncitingFactors,373,[373]
4,PastHistory,373,[373]
...,...,...,...
698,风热侵袭,337,[337]
699,饮酒,373,[373]
700,高脂血症,373,[373]
701,高血压,373,[373]


### 保存leiden算法运行结果

In [24]:
#data_leiden02
data_leiden.to_csv("./outData/dataAll_leiden.csv")


## 标签传播

### 创建具有中医辨证属性的图投影

In [13]:
cypher_="CALL gds.graph.project('labelPropagationWeightDirectedGraph01',"+node_typesStr+","+rel_typeStr+",{nodeProperties: '中医辨证'})"

cypher_

NameError: name 'node_typesStr' is not defined

In [7]:
#tinnitusAllgraph03
cypher_="CALL gds.graph.project('labelPropagationWeightDirectedGraph','Patient','has_syndrome'"+",{nodeProperties: '中医辨证'})"

cypher_

"CALL gds.graph.project('labelPropagationWeightDirectedGraph','Patient','has_syndrome',{nodeProperties: '中医辨证'})"

In [9]:
graph.run(cypher_)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Patient: {label: 'Patient', properties: {中医辨证: {defaultValue: null, property: '\u4e2d\u533b\u8fa8\u8bc1'}}}}","{has_syndrome: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'has_syndrome', properties: {}}}",labelPropagationWeightDirectedGraph,595,0,15


### 运行标签传播算法

In [11]:
'''
relationshipWeightProperty: 'weight',
'''
cypher_="CALL gds.labelPropagation.stream('labelPropagationWeightDirectedGraph',{ seedProperty: '中医辨证' }) YIELD nodeId, communityId AS communityId RETURN gds.util.asNode(nodeId).name AS name, communityId ORDER BY communityId, name"

data_labelPropagation = graph.run(cypher_).to_data_frame()
data_labelPropagation

,name,communityId
0,患者398,0
1,患者399,0
2,患者400,0
3,患者401,0
4,患者402,0
...,...,...
590,患者226,4
591,患者227,4
592,患者228,4
593,患者229,4


In [12]:
data_labelPropagation["communityId"].unique(),len(data_labelPropagation["communityId"])

(array([0, 1, 2, 3, 4], dtype=int64), 595)

### 保存标签传播算法运行结果

In [54]:
data_labelPropagation.to_csv("./outData/data_labelPropagation.csv")


# 节点相似性

In [ ]:
#属性
'''
relationshipWeightProperty:'weight'，要用作权重的关系属性的名称。如果未指定，则该算法运行不加权。
similarityCutoff:0.3,结果中出现的相似度分数的下限。值必须在0和1之间。
topN:10，返回N个最大相似度得分
nodeLabels:使用给定的节点标签筛选命名图。
'''

### 有权重

In [1]:
#权重 5foldAllDirectedGraph02
cypher_="CALL gds.alpha.nodeSimilarity.filtered.stream('5foldAllweightDirectedGraph01',{sourceNodeFilter:'Patient',targetNodeFilter:'Patient'}) YIELD node1, node2, similarity RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity ORDER BY similarity DESCENDING, Person1, Person2"

data_nodeSimilarity = graph.run(cypher_).to_data_frame()
data_nodeSimilarity

NameError: name 'graph' is not defined

In [18]:
#权重 5foldAllDirectedGraph02
cypher_="CALL gds.nodeSimilarity.stream('5foldAllweightDirectedGraph01',{relationshipWeightProperty:'weight'}) YIELD node1, node2, similarity RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity ORDER BY similarity DESCENDING, Person1, Person2"

data_nodeSimilarity = graph.run(cypher_).to_data_frame()
data_nodeSimilarity

,Person1,Person2,similarity
0,多梦,眠浅易醒,0.586801
1,眠浅易醒,多梦,0.586801
2,呕吐,恶心,0.551969
3,恶心,呕吐,0.551969
4,入睡困难,眠浅易醒,0.538912
...,...,...,...
785,外伤,多梦,0.000233
786,耳外伤,蝉鸣声,0.000200
787,耳外伤,苔黄,0.000190
788,耳外伤,苔薄,0.000149


### 无权重

In [17]:
#无权重dataAllgraph02、5foldAllnoweightDirectedGraph05
cypher_="CALL gds.nodeSimilarity.stream('5foldAllweightDirectedGraph01') YIELD node1, node2, similarity RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity ORDER BY similarity DESCENDING, Person1, Person2"

data_nodeSimilarity = graph.run(cypher_).to_data_frame()
data_nodeSimilarity

,Person1,Person2,similarity
0,患者400,患者410,0.987261
1,患者400,患者428,0.987261
2,患者400,患者439,0.987261
3,患者400,患者521,0.987261
4,患者401,患者437,0.987261
...,...,...,...
10115,872,患者400,0.006410
10116,872,患者401,0.006410
10117,872,患者403,0.006410
10118,872,患者404,0.006410


In [138]:
#5折无权重 5foldAllDirectedGraph01、5foldAllDirectedGraph02、5foldAllnoweightDirectedGraph04
cypher_="CALL gds.nodeSimilarity.stream('5foldAllnoweightNumberSyndromeDirectedGraph04') YIELD node1, node2, similarity RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity ORDER BY similarity DESCENDING, Person1, Person2"

data_nodeSimilarity = graph.run(cypher_).to_data_frame()
data_nodeSimilarity

,Person1,Person2,similarity
0,患者1,患者102,1.000000
1,患者1,患者58,1.000000
2,患者100,患者14,1.000000
3,患者100,患者62,1.000000
4,患者100,患者63,1.000000
...,...,...,...
5355,耳外伤,数脉,0.006061
5356,耳外伤,眠浅易醒,0.005348
5357,耳外伤,苔薄,0.004525
5358,耳外伤,蝉鸣声,0.004255


## 保存节点相似性运行结果

In [366]:
data_nodeSimilarity.to_csv("./outData/5折_5dataAll_nodeSimilarity.csv")


In [40]:
data_nodeSimilarity.to_csv("./outData/dataAll_nodeSimilarity权重.csv")


# 共同邻居算法

In [60]:
#读取训练集数据

trainData = pd.read_csv(r"D:\学习资料\项目资料\耳鸣中医知识图谱\neo4j\原始数据\新数据\五折训练集5.csv")
# trainData = pd.read_csv(r"./data/五折训练集4.csv",encoding="gbk")
trainData=trainData.drop("Unnamed: 0",axis=1)
trainData


<IPython.core.display.Javascript object>

,患者,性别,发病节气,劳累,噪音,耳机使用,锻炼,压力,感染,抽烟,...,INR,APTT,FIB,D-二聚体,TT,年龄,病程（月）,TEQ,THI,VAS
0,患者0,女,大寒,0,0,0,0,0,0,0,...,0.97,35.3,2.39,0.19,18.3,45.0,7.0,10,18,6.0
1,患者1,男,大寒,0,0,0,0,0,0,0,...,1.03,34.7,2.85,0.36,18.6,75.0,36.0,8,22,3.0
2,患者2,男,立春,0,0,0,0,0,0,0,...,1.13,38.8,2.79,0.34,19.5,74.0,1.0,12,42,8.0
3,患者3,女,立春,0,0,0,0,0,感染,0,...,1.16,23.6,5.66,0.71,14.7,72.0,0.3,8,24,6.0
4,患者4,女,大寒,劳累,0,0,0,0,0,0,...,0.95,25.1,3.17,0.32,15.6,50.0,1.0,18,78,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,患者1260,女,0,0,0,耳机使用,0,压力,0,0,...,NaN,NaN,NaN,NaN,NaN,36.0,5.0,19,60,8.0
1008,患者1261,女,0,0,0,耳机使用,锻炼,压力,0,0,...,NaN,NaN,NaN,NaN,NaN,52.0,1.0,14,26,7.0
1009,患者1262,女,0,0,0,0,0,压力,0,0,...,NaN,NaN,NaN,NaN,NaN,29.0,6.0,17,44,9.0
1010,患者1263,女,0,0,0,0,0,压力,0,0,...,NaN,NaN,NaN,NaN,NaN,33.0,108.0,16,46,9.0


In [61]:
#读取测试集数据
testData = pd.read_csv(r"D:\学习资料\项目资料\耳鸣中医知识图谱\neo4j\原始数据\新数据\五折测试集5.csv")
# testData = pd.read_csv(r"./data/五折测试集4.csv")
testData=testData.drop("Unnamed: 0",axis=1)
testData

<IPython.core.display.Javascript object>

,患者,性别,发病节气,劳累,噪音,耳机使用,锻炼,压力,感染,抽烟,...,INR,APTT,FIB,D-二聚体,TT,年龄,病程（月）,TEQ,THI,VAS
0,患者7,男,雨水,0,0,0,0,0,0,0,...,1.06,32.4,2.35,0.16,19.0,30.0,1.0,6,14,6.0
1,患者9,女,立春,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,66.0,1.0,18,68,9.0
2,患者11,男,惊蛰,0,0,0,0,0,0,0,...,NaN,NaN,3.30,0.01,18.0,36.0,1.0,12,78,8.0
3,患者16,男,春分,0,0,0,0,0,0,0,...,NaN,NaN,1.45,NaN,22.1,26.0,180.0,5,12,2.0
4,患者20,男,清明,0,0,0,0,0,0,0,...,NaN,NaN,0.58,NaN,24.9,40.0,0.1,8,32,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,患者1246,女,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,48.0,6.0,7,12,3.0
249,患者1248,男,0,0,0,0,0,压力,0,抽烟,...,NaN,NaN,NaN,NaN,NaN,47.0,60.0,10,4,7.0
250,患者1249,女,0,0,噪音,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,62.0,36.0,12,8,9.0
251,患者1255,男,0,0,噪音,0,0,0,0,抽烟,...,NaN,NaN,NaN,NaN,NaN,36.0,24.0,7,10,3.0


In [62]:
#读取所有数据
allData = pd.read_csv(r"D:\学习资料\项目资料\耳鸣中医知识图谱\neo4j\原始数据\新数据\知识图谱新数据.csv",encoding="gbk")
allData

<IPython.core.display.Javascript object>

,患者,性别,发病节气,劳累,噪音,耳机使用,锻炼,压力,感染,抽烟,...,INR,APTT,FIB,D-二聚体,TT,年龄,病程（月）,TEQ,THI,VAS
0,患者0,女,大寒,0,0,0,0,0,0,0,...,0.97,35.3,2.39,0.19,18.3,45.0,7.0,10,18,6.0
1,患者1,男,大寒,0,0,0,0,0,0,0,...,1.03,34.7,2.85,0.36,18.6,75.0,36.0,8,22,3.0
2,患者2,男,立春,0,0,0,0,0,0,0,...,1.13,38.8,2.79,0.34,19.5,74.0,1.0,12,42,8.0
3,患者3,女,立春,0,0,0,0,0,感染,0,...,1.16,23.6,5.66,0.71,14.7,72.0,0.3,8,24,6.0
4,患者4,女,大寒,劳累,0,0,0,0,0,0,...,0.95,25.1,3.17,0.32,15.6,50.0,1.0,18,78,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,患者1260,女,0,0,0,耳机使用,0,压力,0,0,...,NaN,NaN,NaN,NaN,NaN,36.0,5.0,19,60,8.0
1261,患者1261,女,0,0,0,耳机使用,锻炼,压力,0,0,...,NaN,NaN,NaN,NaN,NaN,52.0,1.0,14,26,7.0
1262,患者1262,女,0,0,0,0,0,压力,0,0,...,NaN,NaN,NaN,NaN,NaN,29.0,6.0,17,44,9.0
1263,患者1263,女,0,0,0,0,0,压力,0,0,...,NaN,NaN,NaN,NaN,NaN,33.0,108.0,16,46,9.0


In [13]:
data_Syndrome1,result_Syndrome

([], [])

In [81]:
data_Syndrome1

[None, None]

## 两两计算节点共同邻居数量

### 有权重

In [32]:
data_Syndrome1=[]
data_Syndrome1_=[]
data_Syndrome2_=[]
result_commonNeighbors_=[]
patients1=[]
patients2=[]
result_Syndrome=[]
#训练集trainData["患者"]
label='中医辨证'
# label='二十五音'
#所有数据
for patientName1 in testData["患者"]:
    data_Syndrome2=[]
    result_commonNeighbors=[]
    patient1_Syndrome=graph.nodes.match('Patient', name=f'{patientName1}').first()[f'{label}']
    data_Syndrome1.append(patient1_Syndrome)
    for patientName2 in trainData["患者"]:
        if patientName1 != patientName2:
            cypher_="MATCH (p1:Patient {name:\'"+f'{patientName1}'+"\'}) MATCH (p2:Patient {name: \'"+f'{patientName2}'+"\'}) RETURN gds.alpha.linkprediction.commonNeighbors(p1, p2,{relationshipQuery: \'weight\'}) AS score"
            data_commonNeighbors = graph.run(cypher_).to_data_frame()
            patient2_Syndrome=graph.nodes.match('Patient', name=f'{patientName2}').first()[f'{label}']
            data_Syndrome2.append(patient2_Syndrome)
            data_Syndrome1_.append(patient1_Syndrome)
            data_Syndrome2_.append(patient2_Syndrome)
            result_commonNeighbors_.append(data_commonNeighbors["score"][0])
            result_commonNeighbors.append(data_commonNeighbors["score"][0])
            patients1.append(patientName1)
            patients2.append(patientName2)
    commonNeighbors_result1=getMaxcommonNeighborsSyndrome(data_Syndrome2,result_commonNeighbors)
    maxCommonNeighborsSyndrome=max_list(commonNeighbors_result1['Patient2证型'].values.tolist())
    print("maxCommonNeighborsSyndrome",maxCommonNeighborsSyndrome)
    result_Syndrome.append(maxCommonNeighborsSyndrome)


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 肝火上扰


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 肝火上扰


<IPython.core.display.Javascript object>

...

maxCommonNeighborsSyndrome 肝火上扰


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 肝火上扰


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 肝火上扰


### 无权重

In [11]:
data_Syndrome1=[]
data_Syndrome1_=[]
data_Syndrome2_=[]
result_commonNeighbors_=[]
patients1=[]
patients2=[]
result_Syndrome=[]
#训练集trainData["患者"]
label='中医辨证'
# label='二十五音'
#所有数据
for patientName1 in ['患者1']:
# for patientName1 in testData["患者"]:
    data_Syndrome2=[]
    result_commonNeighbors=[]
    patient1_Syndrome=graph.nodes.match('Patient', name=f'{patientName1}').first()[f'{label}']
    data_Syndrome1.append(patient1_Syndrome)
    for patientName2 in trainData["患者"]:
        if patientName1 != patientName2:
            cypher_="MATCH (p1:Patient {name:\'"+f'{patientName1}'+"\'}) MATCH (p2:Patient {name: \'"+f'{patientName2}'+"\'}) RETURN gds.alpha.linkprediction.commonNeighbors(p1, p2) AS score"
            data_commonNeighbors = graph.run(cypher_).to_data_frame()
            patient2_Syndrome=graph.nodes.match('Patient', name=f'{patientName2}').first()[f'{label}']
            data_Syndrome2.append(patient2_Syndrome)
            data_Syndrome1_.append(patient1_Syndrome)
            data_Syndrome2_.append(patient2_Syndrome)
            result_commonNeighbors_.append(data_commonNeighbors["score"][0])
            result_commonNeighbors.append(data_commonNeighbors["score"][0])
            patients1.append(patientName1)
            patients2.append(patientName2)
    commonNeighbors_result1=getMaxcommonNeighborsSyndrome(data_Syndrome2,result_commonNeighbors)
    maxCommonNeighborsSyndrome=max_list(commonNeighbors_result1['Patient2证型'].values.tolist())
    print("maxCommonNeighborsSyndrome",maxCommonNeighborsSyndrome)
    result_Syndrome.append(maxCommonNeighborsSyndrome)


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 肾精亏虚


In [12]:
data_commonNeighbors

,score
0,8.0


### AA算法

In [39]:
data_Syndrome1=[]
data_Syndrome1_=[]
data_Syndrome2_=[]
result_commonNeighbors_=[]
patients1=[]
patients2=[]
result_Syndrome=[]
#训练集trainData["患者"]
label='中医辨证'
# label='二十五音'
#所有数据
for patientName1 in testData["患者"]:
    data_Syndrome2=[]
    result_commonNeighbors=[]
    patient1_Syndrome=graph.nodes.match('Patient', name=f'{patientName1}').first()[f'{label}']
    data_Syndrome1.append(patient1_Syndrome)
    for patientName2 in trainData["患者"]:
        if patientName1 != patientName2:
            cypher_="MATCH (p1:Patient {name:\'"+f'{patientName1}'+"\'}) MATCH (p2:Patient {name: \'"+f'{patientName2}'+"\'}) RETURN gds.alpha.linkprediction.adamicAdar(p1, p2) AS score"
            data_commonNeighbors = graph.run(cypher_).to_data_frame()
            patient2_Syndrome=graph.nodes.match('Patient', name=f'{patientName2}').first()[f'{label}']
            data_Syndrome2.append(patient2_Syndrome)
            data_Syndrome1_.append(patient1_Syndrome)
            data_Syndrome2_.append(patient2_Syndrome)
            result_commonNeighbors_.append(data_commonNeighbors["score"][0])
            result_commonNeighbors.append(data_commonNeighbors["score"][0])
            patients1.append(patientName1)
            patients2.append(patientName2)
    commonNeighbors_result1=getMaxcommonNeighborsSyndrome(data_Syndrome2,result_commonNeighbors)
    maxCommonNeighborsSyndrome=max_list(commonNeighbors_result1['Patient2证型'].values.tolist())
    print("maxCommonNeighborsSyndrome",maxCommonNeighborsSyndrome)
    result_Syndrome.append(maxCommonNeighborsSyndrome)


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 肾精亏虚


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 肾精亏虚


<IPython.core.display.Javascript object>

...

maxCommonNeighborsSyndrome 痰火郁结


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 痰火郁结


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 痰火郁结


### resourceAllocation

In [63]:
data_Syndrome1=[]
data_Syndrome1_=[]
data_Syndrome2_=[]
result_commonNeighbors_=[]
patients1=[]
patients2=[]
result_Syndrome=[]
#训练集trainData["患者"]
label='中医辨证'
# label='二十五音'
#所有数据
for patientName1 in testData["患者"]:
    data_Syndrome2=[]
    result_commonNeighbors=[]
    patient1_Syndrome=graph.nodes.match('Patient', name=f'{patientName1}').first()[f'{label}']
    data_Syndrome1.append(patient1_Syndrome)
    for patientName2 in trainData["患者"]:
        if patientName1 != patientName2:
            cypher_="MATCH (p1:Patient {name:\'"+f'{patientName1}'+"\'}) MATCH (p2:Patient {name: \'"+f'{patientName2}'+"\'}) RETURN gds.alpha.linkprediction.resourceAllocation(p1, p2) AS score"
            data_commonNeighbors = graph.run(cypher_).to_data_frame()
            patient2_Syndrome=graph.nodes.match('Patient', name=f'{patientName2}').first()[f'{label}']
            data_Syndrome2.append(patient2_Syndrome)
            data_Syndrome1_.append(patient1_Syndrome)
            data_Syndrome2_.append(patient2_Syndrome)
            result_commonNeighbors_.append(data_commonNeighbors["score"][0])
            result_commonNeighbors.append(data_commonNeighbors["score"][0])
            patients1.append(patientName1)
            patients2.append(patientName2)
    commonNeighbors_result1=getMaxcommonNeighborsSyndrome(data_Syndrome2,result_commonNeighbors)
    maxCommonNeighborsSyndrome=max_list(commonNeighbors_result1['Patient2证型'].values.tolist())
    print("maxCommonNeighborsSyndrome",maxCommonNeighborsSyndrome)
    result_Syndrome.append(maxCommonNeighborsSyndrome)


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 肾精亏虚


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 脾胃虚弱


<IPython.core.display.Javascript object>

...

maxCommonNeighborsSyndrome 肝火上扰


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 脾胃虚弱


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 脾胃虚弱


In [9]:
data_Syndrome1=[]
data_Syndrome1_=[]
data_Syndrome2_=[]
result_commonNeighbors_=[]
patients1=[]
patients2=[]
result_Syndrome=[]
#训练集trainData["患者"]
# label='中医辨证'
label='二十五音'
#所有数据
for patientName1 in testData["患者"]:
    data_Syndrome2=[]
    result_commonNeighbors=[]
    patient1_Syndrome=graph.nodes.match('Patient', name=f'{patientName1}').first()[f'{label}']
    data_Syndrome1.append(patient1_Syndrome)
    for patientName2 in trainData["患者"]:
        if patientName1 != patientName2:
            cypher_="MATCH (p1:Patient {name:\'"+f'{patientName1}'+"\'}) MATCH (p2:Patient {name: \'"+f'{patientName2}'+"\'}) RETURN gds.alpha.linkprediction.resourceAllocation(p1, p2) AS score"
            data_commonNeighbors = graph.run(cypher_).to_data_frame()
            patient2_Syndrome=graph.nodes.match('Patient', name=f'{patientName2}').first()[f'{label}']
            data_Syndrome2.append(patient2_Syndrome)
            data_Syndrome1_.append(patient1_Syndrome)
            data_Syndrome2_.append(patient2_Syndrome)
            result_commonNeighbors_.append(data_commonNeighbors["score"][0])
            result_commonNeighbors.append(data_commonNeighbors["score"][0])
            patients1.append(patientName1)
            patients2.append(patientName2)
    commonNeighbors_result1=getMaxcommonNeighborsSyndrome(data_Syndrome2,result_commonNeighbors)
    maxCommonNeighborsSyndrome=max_list(commonNeighbors_result1['Patient2证型'].values.tolist())
    print("maxCommonNeighborsSyndrome",maxCommonNeighborsSyndrome)
    result_Syndrome.append(maxCommonNeighborsSyndrome)


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 钛商


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 质徵


<IPython.core.display.Javascript object>

...

maxCommonNeighborsSyndrome 大宫


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 大宫


<IPython.core.display.Javascript object>

maxCommonNeighborsSyndrome 大羽


In [10]:
len(result_Syndrome)

119

## 保存一 保存共同邻居算法前20%出现证型次数最多结果

In [64]:
# commonNeighbors_result=saveResult(testData,result_Syndrome,5,'test','中医辨证','无权重')
# commonNeighbors_result=saveResult(testData,result_Syndrome,1,'test','中医辨证','AA')
commonNeighbors_result=saveResult(testData,result_Syndrome,5,'test','中医辨证','resourceAllocation')


commonNeighbors_result

<IPython.core.display.Javascript object>

,Patient1,Patient1中医辨证,前20出现次数最多
0,患者7,肝火上扰,肾精亏虚
1,患者9,脾胃虚弱,脾胃虚弱
2,患者11,脾胃虚弱,脾胃虚弱
3,患者16,脾胃虚弱,脾胃虚弱
4,患者20,脾胃虚弱,肝火上扰
...,...,...,...
248,患者1246,风热侵袭,风热侵袭
249,患者1248,肝火上扰,肝火上扰
250,患者1249,肝火上扰,肝火上扰
251,患者1255,脾胃虚弱,脾胃虚弱


In [16]:
def saveResult(Data,result_Syndrome,i,Type,label,fileName):
    commonNeighbors_result=pd.DataFrame()
    commonNeighbors_result["Patient1"]=Data["患者"]
    commonNeighbors_result[f"Patient1{label}"]=Data[f"{label}"]
    commonNeighbors_result["前20出现次数最多"]=result_Syndrome
    commonNeighbors_result.to_csv(f"./outData新数据/{fileName}/5折_{i}{Type}{label}_20result.csv")
    # commonNeighbors_result.to_csv(f"./outData/AA共同邻居/5折_{i}{Type}{label}_20result.csv")
    return commonNeighbors_result

In [17]:
def save(Data,fileName,i,Type,label):
    # Data.to_csv(f"./outData/AA共同邻居/5折_{i}{Type}{label}_{fileName}.csv")
    Data.to_csv(f"./outData新数据/{fileName}/5折_{i}{Type}{label}.csv")
    return Data

## 保存二 准确率

In [56]:
syndrome=[]
accuracys=[]
syndrome_accuracys=pd.DataFrame()
for item in commonNeighbors_result["前20出现次数最多"].unique():
    syndrome.append(item)
    accuracy=sum(commonNeighbors_result[commonNeighbors_result["Patient1中医辨证"]==f"{item}"]["Patient1中医辨证"]==commonNeighbors_result[commonNeighbors_result["Patient1中医辨证"]==f"{item}"]["前20出现次数最多"])/len(commonNeighbors_result[commonNeighbors_result["Patient1中医辨证"]==f"{item}"])
    accuracys.append(accuracy)
    print(item)
    print(accuracy)
syndrome_accuracys["测试集证型"]=syndrome
syndrome_accuracys["准确率"]=accuracys
syndrome_accuracys
# save(syndrome_accuracys,"无权重",5,"中医辨证","多分类结果")
# save(syndrome_accuracys,"AA",2,"中医辨证","多分类结果2")
save(syndrome_accuracys,"resourceAllocation",4,"中医辨证","多分类结果")


<IPython.core.display.Javascript object>

肾精亏虚
0.9272727272727272
脾胃虚弱
0.8181818181818182
肝火上扰
0.9122807017543859
痰火郁结
0.723404255319149
风热侵袭
0.8974358974358975


,测试集证型,准确率
0,肾精亏虚,0.927273
1,脾胃虚弱,0.818182
2,肝火上扰,0.912281
3,痰火郁结,0.723404
4,风热侵袭,0.897436


In [12]:
sum(commonNeighbors_result["Patient1二十五音"]==commonNeighbors_result["前20出现次数最多"])/119

0.33613445378151263

In [357]:
#训练集
sum(commonNeighbors_result["Patient1中医辨证"]==commonNeighbors_result["前20出现次数最多"])/476

0.9705882352941176

In [108]:
#测试集

sum(commonNeighbors_result["Patient1中医辨证"]==commonNeighbors_result["前20出现次数最多"])/119

0.8235294117647058

In [28]:
commonNeighbors_result[commonNeighbors_result["Patient1中医辨证"]=="脾胃亏虚"]

,Patient1,Patient1中医辨证,前20出现次数最多
0,患者7,脾胃亏虚,脾胃亏虚
1,患者17,脾胃亏虚,脾胃亏虚
2,患者24,脾胃亏虚,脾胃亏虚
3,患者25,脾胃亏虚,脾胃亏虚
4,患者27,脾胃亏虚,脾胃亏虚
5,患者32,脾胃亏虚,脾胃亏虚
6,患者40,脾胃亏虚,脾胃亏虚
7,患者41,脾胃亏虚,脾胃亏虚
8,患者46,脾胃亏虚,脾胃亏虚
9,患者50,脾胃亏虚,脾胃亏虚


In [ ]:
sum(commonNeighbors_result["Patient1中医辨证"]==commonNeighbors_result["前20出现次数最多"])/119

In [198]:
commonNeighbors_result=pd.read_csv('./outData/前20test_commonNeighbors_result1.csv')
commonNeighbors_result

<IPython.core.display.Javascript object>

,Unnamed: 0,Patient1,Patient1证型,前20%出现次数最多
0,0,患者1,脾胃亏虚,脾胃亏虚
1,1,患者2,脾胃亏虚,脾胃亏虚
2,2,患者3,脾胃亏虚,脾胃亏虚
3,3,患者4,脾胃亏虚,脾胃亏虚
4,4,患者5,脾胃亏虚,脾胃亏虚
...,...,...,...,...
471,471,患者591,肾精亏损,肾精亏损
472,472,患者592,肾精亏损,肾精亏损
473,473,患者593,肾精亏损,肾精亏损
474,474,患者594,肾精亏损,肾精亏损


## 保存共同邻居算法运行结果

In [219]:
#测试集
commonNeighbors_result=pd.DataFrame()
commonNeighbors_result["Patient1"]=patients1
commonNeighbors_result["Patient1证型"]=data_Syndrome1_
commonNeighbors_result["Patient2"]=patients2
commonNeighbors_result["Patient2证型"]=data_Syndrome2_
commonNeighbors_result["共同邻居数量"]=result_commonNeighbors_

commonNeighbors_result.to_csv("./outData/5折_1Alltest_commonNeighbors_20result.csv")
commonNeighbors_result

<IPython.core.display.Javascript object>

,Patient1,Patient1证型,Patient2,Patient2证型,共同邻居数量
0,患者7,脾胃亏虚,患者1,脾胃亏虚,3.0
1,患者7,脾胃亏虚,患者2,脾胃亏虚,6.0
2,患者7,脾胃亏虚,患者3,脾胃亏虚,6.0
3,患者7,脾胃亏虚,患者4,脾胃亏虚,5.0
4,患者7,脾胃亏虚,患者5,脾胃亏虚,6.0
...,...,...,...,...,...
56639,患者587,肾精亏损,患者591,肾精亏损,6.0
56640,患者587,肾精亏损,患者592,肾精亏损,5.0
56641,患者587,肾精亏损,患者593,肾精亏损,3.0
56642,患者587,肾精亏损,患者594,肾精亏损,4.0


## 计算共同邻居数量前20节点出现次数最多的证型

In [8]:
def getMaxcommonNeighborsSyndrome(data_Syndrome2,result_commonNeighbors):
    commonNeighbors_result=pd.DataFrame()
    # commonNeighbors_result["Patient1证型"]=data_Syndrome1
    # commonNeighbors_result["Patient2"]=patients2
    commonNeighbors_result["Patient2证型"]=data_Syndrome2
    commonNeighbors_result["共同邻居数量"]=result_commonNeighbors
    commonNeighbors_result=commonNeighbors_result.sort_values(by=['共同邻居数量'],ascending=False)
    commonNeighbors_result1=commonNeighbors_result[:20]
    # maxSyndrome=max_list(commonNeighbors_result1['Patient2证型'].values.tolist())

    return commonNeighbors_result1


In [91]:
#读取测试集数据
commonNeighbors_testresult = pd.read_csv(r"./outData/dataAll_commonNeighbors_testresult.csv",encoding="gbk")
commonNeighbors_testresult

<IPython.core.display.Javascript object>

,Unnamed: 0,Patient1,Patient1证型,前20%出现次数最多证型
0,0,患者471,0,3
1,1,患者328,1,2
2,2,患者355,1,0
3,3,患者564,0,0
4,4,患者326,1,1
...,...,...,...,...
174,174,患者475,0,4
175,175,患者189,4,1
176,176,患者31,3,3
177,177,患者195,4,1


In [95]:
sum(commonNeighbors_testresult["Patient1证型"]==commonNeighbors_testresult["前20%出现次数最多证型"])/179

0.4692737430167598

# 准确率计算

In [24]:
data1 = pd.read_csv(r"D:\学习资料\项目资料\耳鸣中医知识图谱\neo4j\知识图谱耳鸣数据.csv",encoding="gbk")
data1

<IPython.core.display.Javascript object>

,患者,性别,发病节气,噪音,劳累,压力,感染,外伤,高血压,糖尿病,...,APTT,FIB,D-二聚体,TT,频率,TEQ,THI,VAS,病程(月),年龄
0,患者1,女,秋分,0,0,0,0,0,高血压,0,...,27.615,2.696,0.334,18.399,4315,13,46,6,60.0,63
1,患者2,女,大雪,0,0,0,0,0,高血压,0,...,31.800,3.470,0.250,17.300,2243,8,18,6,1.0,45
2,患者3,女,冬至,0,0,0,0,0,0,0,...,25.900,2.940,0.570,17.700,682,10,14,2,0.1,69
3,患者4,女,小寒,0,0,0,0,0,高血压,0,...,20.900,2.696,0.334,18.399,6631,13,44,5,0.2,57
4,患者5,男,小寒,0,0,0,0,0,0,0,...,27.615,2.696,0.334,18.399,1352,10,34,4,1.0,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,患者591,男,秋分,噪音,0,0,0,0,0,0,...,27.615,2.696,0.334,18.399,646,13,24,5,5.0,52
591,患者592,男,秋分,0,0,压力,感染,0,0,0,...,27.615,2.696,0.334,18.399,1352,11,4,4,216.0,37
592,患者593,女,秋分,0,0,压力,0,0,0,0,...,27.615,2.696,0.334,18.399,6165,8,2,6,24.0,54
593,患者594,男,白露,0,0,0,0,0,高血压,0,...,25.200,2.440,0.190,17.600,2644,11,34,6,3.0,63


## 读取Louvain运行结果


In [21]:
#data_Louvain02
data2 = pd.read_csv(r"./outData/dataAll_Louvain权重.csv")
columns=data2.columns
data2,columns

<IPython.core.display.Javascript object>

(     Unnamed: 0                        name  communityId  \
 0             0                CoatedTongue          117   
 1             1      ConditionOfPersistence           42   
 2             2  DiagnosisofWesternMedicine          104   
 3             3             IncitingFactors           28   
 4             4                 PastHistory           34   
 ..          ...                         ...          ...   
 722         722                        风热侵袭           77   
 723         723                          饮酒           41   
 724         724                        高脂血症           37   
 725         725                         高血压           35   
 726         726                         鸣笛声           49   
 
     intermediateCommunityIds  
 0                      [117]  
 1                       [42]  
 2                      [104]  
 3                       [28]  
 4                       [34]  
 ..                       ...  
 722                     [77]  
 723      


## 读取leiden运行结果


In [32]:
#data_leiden01
data2 = pd.read_csv(r"./outData/dataAll_leiden.csv")
columns=data2.columns
print(columns)
data2


<IPython.core.display.Javascript object>

Index(['Unnamed: 0', 'name', 'communityId', 'intermediateCommunityIds'], dtype='object')


,Unnamed: 0,name,communityId,intermediateCommunityIds
0,0,CoatedTongue,373,[373]
1,1,ConditionofPersistence,373,[373]
2,2,DiagnosisofWesternMedicine,373,[373]
3,3,IncitingFactors,373,[373]
4,4,PastHistory,373,[373]
...,...,...,...,...
698,698,风热侵袭,337,[337]
699,699,饮酒,373,[373]
700,700,高脂血症,373,[373]
701,701,高血压,373,[373]


## 读取标签传播算法运行

In [68]:
data2 = pd.read_csv(r"./outData/data_labelPropagation.csv")
columns=data2.columns
print(columns)
data2


<IPython.core.display.Javascript object>

Index(['Unnamed: 0', 'name', 'communityId'], dtype='object')


,Unnamed: 0,name,communityId
0,0,患者398,0
1,1,患者399,0
2,2,患者401,0
3,3,患者402,0
4,4,患者403,0
...,...,...,...
590,590,患者475,987
591,591,患者189,988
592,592,患者31,989
593,593,患者195,990


In [22]:
#只获取name为患者的行
data2_=pd.DataFrame(columns=columns)

j=0
for i in range(len(data2)):
    res = re.findall('患者', data2['name'][i])
    if res!=[] and res[0] == '患者':
        data2_.loc[j]=data2.loc[i]
        j=j+1
data2_

<IPython.core.display.Javascript object>

,Unnamed: 0,name,communityId,intermediateCommunityIds
0,66,患者1,285,[285]
1,67,患者10,489,[489]
2,68,患者100,284,[284]
3,69,患者101,528,[528]
4,70,患者102,468,[468]
...,...,...,...,...
590,656,患者95,431,[431]
591,657,患者96,506,[506]
592,658,患者97,430,[430]
593,659,患者98,691,[691]


In [25]:
#提取各个耳鸣分类的患者名
pwkx=data1[data1["中医辨证"] == "脾胃亏虚"]["患者"].values
sjks=data1[data1["中医辨证"] == "肾精亏损"]["患者"].values
thyj=data1[data1["中医辨证"] == "痰火郁结"]["患者"].values
frqx=data1[data1["中医辨证"] == "风热侵袭"]["患者"].values
ghsr=data1[data1["中医辨证"] == "肝火上扰"]["患者"].values
# pwkx,sjks,thyj,frqx,ghsr

## 计算社区里每个耳鸣分类的占比

In [26]:
#计算社区里每个耳鸣分类的占比脾胃亏虚肾精亏损痰火郁结肝火上扰
communityIds=data2_["communityId"].unique()
community_datas=[]
for community_id in communityIds:
    community_pwkx=[]
    community_sjks=[]
    community_thyj=[]
    community_frqx=[]
    community_ghsr=[]
    for i in range(len(data2_)):
        patient_item=data2_["name"][i]
        communityId=data2_["communityId"][i]
        if communityId==community_id:
            if patient_item in pwkx:
                community_pwkx.append(patient_item)
                # print(patient_item,"脾胃亏虚")
            if patient_item in sjks:
                community_sjks.append(patient_item)
                # print(patient_item,"肾精亏损")
            if patient_item in thyj:
                community_thyj.append(patient_item)
                # print(patient_item,"痰火郁结")
            if patient_item in frqx:
                community_frqx.append(patient_item)
                # print(patient_item,"风热侵袭")
            if patient_item in ghsr:
                community_ghsr.append(patient_item)
                # print(patient_item,"肝火上扰")
    community_data=[community_id,len(community_pwkx),len(community_sjks),len(community_thyj),len(community_frqx),len(community_ghsr)]            
    sum_community=len(community_pwkx)+len(community_sjks)+len(community_thyj)+len(community_frqx)+len(community_ghsr)
    print(sum_community)
    # print(community_id,sum_community,"脾胃亏虚",len(community_pwkx),"肾精亏损",len(community_sjks),"痰火郁结",len(community_thyj),"风热侵袭",len(community_frqx),"肝火上扰",len(community_ghsr))
    community_datas.append(community_data)
print(community_datas)
    

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


## 保存Louvain准确率结果


In [27]:
community_result=pd.DataFrame(data=community_datas,columns=['communityId','脾胃亏虚','肾精亏损','痰火郁结','风热侵袭','肝火上扰'])
community_result.to_csv("./outData/dataAll_louvain_result权重.csv")
community_result

<IPython.core.display.Javascript object>

,communityId,脾胃亏虚,肾精亏损,痰火郁结,风热侵袭,肝火上扰
0,285,1,0,0,0,0
1,489,1,0,0,0,0
2,284,1,0,0,0,0
3,528,1,0,0,0,0
4,468,1,0,0,0,0
...,...,...,...,...,...,...
590,431,1,0,0,0,0
591,506,1,0,0,0,0
592,430,1,0,0,0,0
593,691,1,0,0,0,0


## 保存leiden准确率结果


In [35]:
#leiden_result
community_result=pd.DataFrame(data=community_datas,columns=['communityId','脾胃亏虚','肾精亏损','痰火郁结','风热侵袭','肝火上扰'])
community_result.to_csv("./outData/dataAll_leiden_result.csv")
community_result

<IPython.core.display.Javascript object>

,communityId,脾胃亏虚,肾精亏损,痰火郁结,风热侵袭,肝火上扰
0,248,174,0,0,0,0
1,337,0,0,0,57,0
2,373,0,0,76,0,91
3,636,0,197,0,0,0


## 保存标签传播准确率结果


In [69]:
labelPropagation_result=pd.DataFrame(data=community_datas,columns=['communityId','脾胃亏虚','肾精亏损','痰火郁结','风热侵袭','肝火上扰'])
labelPropagation_result.to_csv("./outData/labelPropagation_result.csv")
labelPropagation_result

<IPython.core.display.Javascript object>

,communityId,脾胃亏虚,肾精亏损,痰火郁结,风热侵袭,肝火上扰
0,0,0,138,0,0,0
1,1,0,0,0,0,63
2,2,0,0,53,0,0
3,3,122,0,0,0,0
4,4,0,0,0,40,0
...,...,...,...,...,...,...
179,987,0,1,0,0,0
180,988,0,0,0,1,0
181,989,1,0,0,0,0
182,990,0,0,0,1,0


# 节点相似度排序

In [22]:
#读取训练集数据
# trainData = pd.read_csv(r"D:\学习资料\项目资料\耳鸣中医知识图谱\neo4j\知识图谱训练数据.csv",encoding="gbk")
trainData = pd.read_csv(r"./data/五折训练集1.csv")
trainData=trainData.drop("Unnamed: 0",axis=1)
#读取测试集数据
# testData = pd.read_csv(r"D:\学习资料\项目资料\耳鸣中医知识图谱\neo4j\知识图谱测试数据.csv",encoding="gbk")
testData = pd.read_csv(r"./data/五折测试集1.csv")
testData=testData.drop("Unnamed: 0",axis=1)
trainData,testData

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(        患者 性别 发病节气  噪音 劳累  压力  感染 外伤  高血压 糖尿病  ...    APTT    FIB  D-二聚体  \
 0      患者1  女   秋分   0  0   0   0  0  高血压   0  ...  27.615  2.696  0.334   
 1      患者2  女   大雪   0  0   0   0  0  高血压   0  ...  31.800  3.470  0.250   
 2      患者3  女   冬至   0  0   0   0  0    0   0  ...  25.900  2.940  0.570   
 3      患者4  女   小寒   0  0   0   0  0  高血压   0  ...  20.900  2.696  0.334   
 4      患者5  男   小寒   0  0   0   0  0    0   0  ...  27.615  2.696  0.334   
 ..     ... ..  ...  .. ..  ..  .. ..  ...  ..  ...     ...    ...    ...   
 471  患者591  男   秋分  噪音  0   0   0  0    0   0  ...  27.615  2.696  0.334   
 472  患者592  男   秋分   0  0  压力  感染  0    0   0  ...  27.615  2.696  0.334   
 473  患者593  女   秋分   0  0  压力   0  0    0   0  ...  27.615  2.696  0.334   
 474  患者594  男   白露   0  0   0   0  0  高血压   0  ...  25.200  2.440  0.190   
 475  患者595  女   立冬   0  0   0   0  0  高血压   0  ...  27.615  2.696  0.334   
 
          TT    频率 TEQ THI VAS  病程(月)  年龄  
 0    18.399  4315  13  46   6

In [368]:
# data3 = pd.read_csv(r"./outData无权重/dataAll_nodeSimilarity权重.csv")
data3 = pd.read_csv(r"./outData/5折_5dataAll_nodeSimilarity.csv")
# data3 =pd.read_csv(r"D:\学习资料\项目资料\耳鸣中医知识图谱\neo4j\outData\data_nodeSimilarity.csv",encoding="gbk")
#,encoding="gbk"
columns=data3.columns
print(columns)
data3

<IPython.core.display.Javascript object>

Index(['Unnamed: 0', 'Person1', 'Person2', 'similarity'], dtype='object')


,Unnamed: 0,Person1,Person2,similarity
0,0,患者1,患者102,1.000000
1,1,患者10,患者118,1.000000
2,2,患者10,患者127,1.000000
3,3,患者10,患者143,1.000000
4,4,患者100,患者14,1.000000
...,...,...,...,...
6785,6785,耳外伤,入睡困难,0.006061
6786,6786,耳外伤,数脉,0.006061
6787,6787,耳外伤,眠浅易醒,0.005348
6788,6788,耳外伤,苔薄,0.004525


In [27]:
data3=data_nodeSimilarity
columns=data3.columns
data_=getNameRow(data3,columns)
data_

<IPython.core.display.Javascript object>

,Person1,Person2,similarity
0,患者400,患者410,0.987261
1,患者400,患者428,0.987261
2,患者400,患者439,0.987261
3,患者400,患者521,0.987261
4,患者401,患者437,0.987261
...,...,...,...
5945,患者225,患者192,0.833333
5946,患者225,患者193,0.833333
5947,患者225,患者200,0.833333
5948,患者225,患者218,0.833333


In [ ]:
'患者2' in testData["患者"].values

True

In [ ]:
len(data_["Person2"].unique())

517

In [ ]:
for item in data_[data_["Person1"]=="患者2"]["Person2"]:
    if item in testData["患者"].values:
        print("item")

In [29]:
for item in data_["Person1"]:
    person2=data_[data_["Person1"]=="item"]["Person2"]
    if item in testData["患者"].values:
        for item2 in person2:
            if  item2 in trainData["患者"].values:
                print("item",item)

In [ ]:
#训练集

similarity_Syndrome=[]
patient_Syndrome=[]
patient1=[]
patient2=[]
for item in data_["Person1"].unique():
    if item in trainData["患者"].values:
        # print("item",item)
        node_Syndrome=graph.nodes.match('Patient', name=f'{item}').first()['中医辨证']
        patient_Syndrome.append(node_Syndrome)
        patient1.append(item)
        # print("item",item,patient_Syndrome)
        similarity_datas=[]
        Patient_similarity=data_[data_["Person1"] == f"{item}"]
        for item1 in Patient_similarity["Person2"]:
            if item1 in trainData["患者"].values:
                patient2.append(item1)
                data_Syndrome=graph.nodes.match('Patient', name=f'{item1}').first()['中医辨证']
                similarity_datas.append(data_Syndrome)
                # print(item1,data_Syndrome)
        max_Syndrome=max_list(similarity_datas)
        similarity_Syndrome.append(max_Syndrome)
# similarity_datas=[]
# similarity_Syndrome=[]
# Patient_similarity=data_[data_["Person1"] == '患者1']
# for item1 in Patient_similarity["Person2"]:
#     data_Syndrome=graph.nodes.match('Patient', name=f'{item1}').first()['中医辨证']
#     similarity_datas.append(data_Syndrome)
#     print(item1,data_Syndrome)
# max_Syndrome=max_list(similarity_datas)

similarity_datas数组为空


UnboundLocalError: local variable 'max_str' referenced before assignment

In [370]:
#测试集
similarity_Syndrome=[]
patient_Syndrome=[]
patient1=[]
patient2=[]
for item in data_["Person1"].unique():
    if item in testData["患者"].values:
        # print("item",item)
        node_Syndrome=graph.nodes.match('Patient', name=f'{item}').first()['中医辨证']
        patient_Syndrome.append(node_Syndrome)
        patient1.append(item)
        # print("item",item,patient_Syndrome)
        similarity_datas=[]
        Patient_similarity=data_[data_["Person1"] == f"{item}"]
        for item1 in Patient_similarity["Person2"]:
            if item1 in trainData["患者"].values:
                # print(item1)
                patient2.append(item1)
                data_Syndrome=graph.nodes.match('Patient', name=f'{item1}').first()['中医辨证']
                similarity_datas.append(data_Syndrome)
        # print("similarity_datas",similarity_datas)
        max_Syndrome=max_list(similarity_datas)
        similarity_Syndrome.append(max_Syndrome)
print(len(similarity_Syndrome))
# similarity_datas=[]
# similarity_Syndrome=[]
# Patient_similarity=data_[data_["Person1"] == '患者1']
# for item1 in Patient_similarity["Person2"]:
#     data_Syndrome=graph.nodes.match('Patient', name=f'{item1}').first()['中医辨证']
#     similarity_datas.append(data_Syndrome)
#     print(item1,data_Syndrome)
# max_Syndrome=max_list(similarity_datas)

similarity_datas数组为空


UnboundLocalError: local variable 'max_str' referenced before assignment

In [321]:
from collections import Counter

a=Counter(similarity_datas)
a

Counter()

In [128]:
len(patient1)

179

In [251]:
similarity_datas=[]
similarity_Syndrome=[]
Patient_similarity=data_[data_["Person1"] == '患者445']
for item1 in Patient_similarity["Person2"]:
    
    data_Syndrome=graph.nodes.match('Patient', name=f'{item1}').first()['中医辨证']
    similarity_datas.append(data_Syndrome)
    print(item1,data_Syndrome)
max_Syndrome=max_list(similarity_datas)

similarity_Syndrome.append(max_Syndrome)
similarity_datas,similarity_Syndrome

患者442 肾精亏损
患者454 肾精亏损
患者458 肾精亏损
患者462 肾精亏损
患者466 肾精亏损
患者478 肾精亏损
患者486 肾精亏损
患者489 肾精亏损
患者494 肾精亏损
患者522 肾精亏损


(['肾精亏损',
  '肾精亏损',
  '肾精亏损',
  '肾精亏损',
  '肾精亏损',
  '肾精亏损',
  '肾精亏损',
  '肾精亏损',
  '肾精亏损',
  '肾精亏损'],
 ['肾精亏损'])

In [50]:
data_Syndrome=graph.nodes.match('Patient', name='患者2').first()['中医辨证']
data_Syndrome

3

In [41]:
cypher_="Match (n:Patient{name:'患者1'}) return n.中医辨证 AS Syndrome"
data_Syndrome = graph.run(cypher_).to_data_frame()["Syndrome"][0]
data_Syndrome

3

## 保存前10出现次数最多证型结果

In [208]:
#测试集
similarity_result=pd.DataFrame()
similarity_result["Patient1"]=patient1
similarity_result["证型"]=patient_Syndrome
similarity_result["前10出现次数最多证型"]=similarity_Syndrome
similarity_result.to_csv("./outData/5折_1similarity_trainResult.csv")
similarity_result

<IPython.core.display.Javascript object>

,Patient1,证型,前10出现次数最多证型
0,患者1,脾胃亏虚,脾胃亏虚
1,患者10,脾胃亏虚,脾胃亏虚
2,患者100,脾胃亏虚,脾胃亏虚
3,患者101,脾胃亏虚,脾胃亏虚
4,患者102,脾胃亏虚,脾胃亏虚
...,...,...,...
471,患者586,肾精亏损,肾精亏损
472,患者588,肾精亏损,肾精亏损
473,患者84,脾胃亏虚,脾胃亏虚
474,患者87,脾胃亏虚,脾胃亏虚


In [209]:
correct=np.sum(similarity_result["证型"]==similarity_result["前10出现次数最多证型"])
correct/476

<IPython.core.display.Javascript object>

0.9411764705882353

In [211]:
#测试集
similarity_result=pd.DataFrame()
similarity_result["Patient1"]=patient1
similarity_result["证型"]=patient_Syndrome
similarity_result["前10出现次数最多证型"]=similarity_Syndrome
similarity_result.to_csv("./outData/5折_1similarity_testResult.csv")
similarity_result

<IPython.core.display.Javascript object>

,Patient1,证型,前10出现次数最多证型
0,患者10,脾胃亏虚,脾胃亏虚
1,患者118,脾胃亏虚,脾胃亏虚
2,患者148,脾胃亏虚,脾胃亏虚
3,患者152,脾胃亏虚,脾胃亏虚
4,患者153,脾胃亏虚,脾胃亏虚
...,...,...,...
114,患者53,脾胃亏虚,脾胃亏虚
115,患者550,肾精亏损,肾精亏损
116,患者552,肾精亏损,肾精亏损
117,患者87,脾胃亏虚,脾胃亏虚


In [212]:
correct=np.sum(similarity_result["证型"]==similarity_result["前10出现次数最多证型"])
correct/119

<IPython.core.display.Javascript object>

0.9411764705882353

In [130]:
similarity_testResult = pd.read_csv(r"./outData/similarity_testResult.csv",encoding="gbk")
similarity_testResult

<IPython.core.display.Javascript object>

,Unnamed: 0,Patient1,证型,前10出现次数最多证型
0,0,患者267,2,2
1,1,患者269,2,1
2,2,患者288,2,2
3,3,患者303,2,2
4,4,患者311,1,1
...,...,...,...,...
174,174,患者367,1,1
175,175,患者375,1,1
176,176,患者385,1,1
177,177,患者388,1,1


In [135]:
correct=np.sum(similarity_testResult["证型"]==similarity_testResult["前10出现次数最多证型"])
correct

<IPython.core.display.Javascript object>

176

In [132]:
correct/179

0.9832402234636871

# 只获取name为患者的行

In [9]:
#只获取name为患者的行
def getNameRow(df,columns):
    data_=pd.DataFrame(columns=columns)
    j=0
    for i in range(len(df)):
        res = re.findall('患者', data3['Person1'][i])
        if res!=[] and res[0] == '患者':
            data_.loc[j]=df.loc[i]
            j=j+1
    return data_

# 获取出现次数最多的证型


In [10]:
#获取出现次数最多的证型
def max_list(similarity_datas):
    temp = 0
    if len(similarity_datas) !=0:
        for item in similarity_datas:
            if similarity_datas.count(item) > temp:
                max_str = item
                temp = similarity_datas.count(item)
    else:
        print("similarity_datas数组为空")
    return max_str